In [1]:
import os
import csv
import cv2
import numpy as np
import random
from keras.models import Sequential, Model
from keras.layers import Activation, Flatten, Dense, Lambda, Cropping2D, Convolution2D
from keras import backend as K
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


image_paths = []
measurements = []
adjusted_angle = 0.2
data_folder = './GoodData/'
image_folder = data_folder + 'IMG/'

with open(data_folder + 'driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        measurement = float(line[3])

        image_paths.append(line[0].split('\\')[-1])
        measurements.append(measurement)

        image_paths.append(line[1].split('\\')[-1])
        measurements.append(measurement + adjusted_angle)

        image_paths.append(line[2].split('\\')[-1])
        measurements.append(measurement - adjusted_angle)   
    
X_train = np.array(image_paths)
y_train = np.array(measurements)

## SPLIT TRAIN AND VALID DATA
X_train, y_train = shuffle(X_train, y_train)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2) 

C:\Users\van.vu\AppData\Local\Continuum\miniconda2\envs\IntroToTensorFlow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(len(X_valid))

2400


In [7]:
def random_flip(image,steering):
    coin=np.random.randint(0,2)
    if coin==0:
        image, steering = cv2.flip(image,1), -steering
    return image,steering

def random_brightness(image):
    image1 = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = random.uniform(0.3,1.0)    
    image1[:,:,2] = image1[:,:,2]*random_bright
    image1 = cv2.cvtColor(image1,cv2.COLOR_HSV2RGB)
    return image1

def crop_and_resize(image):
    cropped = cv2.resize(image[60:140,:], (64,64))
    return cropped

def preprocess_image(image_path,steering, isTrainImage = True):
    image = cv2.imread(image_folder + image_path)
    cropped_image = crop_and_resize(image) 
    if isTrainImage:
        flip_image, flip_steering = random_flip(cropped_image, steering)
        flip_image = random_brightness(flip_image)
        return flip_image, flip_steering
    else:
        return cropped_image, steering

In [4]:
def getModel():
    model = Sequential()
    model.add(Lambda(lambda x: x /255.0 - 0.5, input_shape=(160,320,3)))
    model.add(Cropping2D(cropping=((50,20),(0,0))))
    model.add(Convolution2D(24,(5,5),strides=(2,2),activation="relu"))
    model.add(Convolution2D(36,(5,5),strides=(2,2),activation="relu"))
    model.add(Convolution2D(48,(5,5),strides=(2,2),activation="relu"))
    model.add(Convolution2D(64,(3,3),activation="relu"))
    model.add(Convolution2D(64,(3,3),activation="relu"))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(1))
    return model

In [5]:
def generator(X_data, y_data, isTrainImages = True, batch_size=32):
    while 1: # Loop forever so the generator never terminates
        images, measurements = shuffle(X_data, y_data)
        
        batch_images = []
        batch_measurements = []
        for i in range(batch_size):
            choice = int(np.random.choice(len(images),1))
            image, measurement = preprocess_image(images[choice], measurements[choice], isTrainImages)
            
            batch_images.append(image)
            batch_measurements.append(measurement)
        
        yield np.array(batch_images), np.array(batch_measurements)
            
            
# compile and train the model using the generator function
train_generator = generator(X_train, y_train)
validation_generator = generator(X_valid, y_valid, False)

model = getModel()

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch= len(X_train), validation_data=validation_generator,
                    nb_val_samples=len(X_valid), nb_epoch=1)

model.save('model.h5')

C:\Users\van.vu\AppData\Local\Continuum\miniconda2\envs\IntroToTensorFlow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\van.vu\AppData\Local\Continuum\miniconda2\envs\IntroToTensorFlow\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., validation_data=<generator..., steps_per_epoch=9600, epochs=1, validation_steps=2400)`


Epoch 1/1
9600/9600 [==============================] - 7184s 748ms/step - loss: 0.0465 - val_loss: 0.0410


In [ ]:
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    #augmented_images.append(image)
    #augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*-1.0)

images.extend(augmented_images)
measurements.extend(augmented_measurements)
    

In [ ]:
print(len(measurements))

In [ ]:



model = Sequential()
model.add(Lambda(lambda x: x /255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50,20),(0,0))))
model.add(Convolution2D(24,(5,5),strides=(2,2),activation="relu"))
model.add(Convolution2D(36,(5,5),strides=(2,2),activation="relu"))
model.add(Convolution2D(48,(5,5),strides=(2,2),activation="relu"))
model.add(Convolution2D(64,(3,3),activation="relu"))
model.add(Convolution2D(64,(3,3),activation="relu"))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
#model.add(Activation('softmax'))

model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=3)

model.save('model.h5')
